In [ ]:
# Install in Google Colab
try:
    import google.colab
    !pip install -q -i https://test.pypi.org/simple/ --extra-index-url https://pypi.org/simple/ quantem-widget
except ImportError:
    pass  # Not in Colab, skip

In [1]:
%load_ext autoreload
%autoreload 2
%env ANYWIDGET_HMR=1

env: ANYWIDGET_HMR=1


[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/bobleesj/quantem.widget/blob/main/notebooks/show3d/show3d_simple.ipynb)

# Show3D — Quick Demo


In [2]:
import numpy as np


def make_focal_series(n_frames=30, size=256):
    """Through-focus series: nanoparticles with Fresnel fringes at edges."""
    y, x = np.mgrid[:size, :size]
    particles = [
        (size * 0.35, size * 0.4, 18, 1.0),  # (cx, cy, radius, Z-contrast)
        (size * 0.65, size * 0.55, 25, 0.7),
        (size * 0.45, size * 0.7, 12, 1.2),
        (size * 0.7, size * 0.3, 15, 0.9),
    ]
    defocus = np.linspace(-60, 60, n_frames)
    frames = np.zeros((n_frames, size, size), dtype=np.float32)
    for f_idx, df in enumerate(defocus):
        frame = np.full((size, size), 0.5)  # uniform background
        for cx, cy, r, z in particles:
            dist = np.sqrt((x - cx) ** 2 + (y - cy) ** 2)
            # In focus: sharp edge
            edge = 1.0 / (1 + np.exp((dist - r) * 2))
            if abs(df) > 3:
                # Defocused: Fresnel fringes at particle edges
                fresnel = np.cos(0.005 * df * (dist - r) ** 2) * np.exp(
                    -((dist - r) ** 2) / (2 * (3 + abs(df) * 0.15) ** 2)
                )
                frame += z * (edge * 0.3 + fresnel * 0.2 * np.sign(df))
            else:
                frame += z * edge * 0.4
        frame += np.random.normal(0, 0.03, (size, size))
        frames[f_idx] = frame
    return frames


focal_stack = make_focal_series()
print(f"Shape: {focal_stack.shape}, range: [{focal_stack.min():.2f}, {focal_stack.max():.2f}]")

Shape: (30, 256, 256), range: [0.18, 1.16]


In [3]:
from quantem.widget import Show3D

defocus_values = np.linspace(-60, 60, 30)
labels = [f"C10={df:.0f} nm" for df in defocus_values]

Show3D(
    focal_stack,
    labels=labels,
    title="Through-Focus Series: Nanoparticles with Fringes",
    cmap="gray",
    pixel_size=0.25,
    fps=8,
)

Show3D(30×256×256, frame=15, cmap=gray)

In [4]:
# Circle ROI centered on a nanoparticle
w_circle = Show3D(
    focal_stack,
    title="Circle ROI on Particle",
    cmap="gray",
    pixel_size=0.25,
)
w_circle

Show3D(30×256×256, frame=15, cmap=gray)

In [5]:
# Square ROI on another particle
w_square = Show3D(
    focal_stack,
    title="Square ROI on Particle",
    cmap="inferno",
    pixel_size=0.25,
)
w_square.set_roi(row=102, col=90, radius=20)
w_square.roi_square(20)
w_square

Show3D(30×256×256, frame=15, cmap=inferno)

In [6]:
# Rectangle ROI spanning a region between two particles
w_rect = Show3D(
    focal_stack,
    title="Rectangle ROI Between Particles",
    cmap="viridis",
    pixel_size=0.25,
)
w_rect.set_roi(row=128, col=128)
w_rect.roi_rectangle(width=80, height=40)
w_rect

Show3D(30×256×256, frame=15, cmap=viridis)

## Inspect Widget State

In [7]:
w_circle.summary()

Circle ROI on Particle
════════════════════════════════
Stack:    30×256×256 (0.25 nm/px)
Frame:    15/29 [15]
Data:     min=0.1768  max=1.156  mean=0.5168
Display:  gray | manual contrast | linear
Playback: 5.0 fps | loop=on | reverse=off | boomerang=off
